In [2]:
!pip install -U nlp

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 8.9 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import nlp
import numpy as np
import scipy as sp
import torch
import transformers

import shap

d:\Pdir\MMFair\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# load a BERT sentiment analysis model
tokenizer = transformers.DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")
model = transformers.DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
).cuda()

d:\Pdir\MMFair\.venv\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\drago\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
d:\Pdir\MMFair\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is depreca

In [5]:
# define a prediction function
def f(x):
    tv = torch.tensor([tokenizer.encode(v, padding="max_length", max_length=500, truncation=True) for v in x]).cuda()
    outputs = model(tv)[0].detach().cpu().numpy()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:, 1])  # use one vs rest logit units
    return val


# build an explainer using a token masker
explainer = shap.Explainer(f, tokenizer)

In [6]:
# explain the model's predictions on IMDB reviews
imdb_train = nlp.load_dataset("imdb")["train"]
shap_values = explainer(imdb_train[:10], fixed_context=1)

Downloading: 100%|██████████| 4.56k/4.56k [00:00<?, ?B/s]


Downloading: 100%|██████████| 84.1M/84.1M [02:17<00:00, 610kB/s] 


Dataset imdb downloaded and prepared to C:\Users\drago\.cache\huggingface\datasets\imdb\plain_text\1.0.0\76cdbd7249ea3548c928bbf304258dab44d09cd3638d9da8d42480d1d1be3743. Subsequent calls will reuse this data.


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
PartitionExplainer explainer:  50%|█████     | 1/2 [00:00<?, ?it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (559 > 512). Running this sequence through the model will result in indexing errors
PartitionExplainer explainer: 11it [01:10,  7.02s/it]              


In [7]:
# plot the first sentence's explanation
shap.plots.text(shap_values[3])

In [9]:
shap_values[0]

.values =
array([ 6.22555266e-03,  4.95740424e-03,  8.13671599e-03,  9.87269546e-02,
        3.59844829e-02, -3.59289979e-02,  2.31135136e-01, -7.65796178e-01,
        2.80987150e-01, -9.22162866e-02, -1.33553635e-01, -3.86777054e-01,
       -3.26663367e-02, -6.95241611e-03, -1.68038737e-01,  1.99822076e-02,
       -1.01849686e-01,  1.16850026e-02,  8.09126345e-02, -1.20725187e-02,
        1.17567825e-02,  3.00642300e-02,  5.26720333e-02, -4.79033799e-04,
       -2.89385135e-02, -4.64906509e-02, -4.33816329e-02, -8.48855391e-02,
        2.10700616e-02,  4.55832427e-02,  5.03637046e-03, -3.75261143e-02,
        3.41863796e-02,  1.11510918e-02, -3.16442087e-02, -1.35135515e-01,
       -3.78324822e-02, -1.54840930e-01,  1.34460777e-03,  5.09540886e-03,
       -5.38427368e-02, -5.43403164e-02, -2.99119303e-03, -3.32989628e-02,
       -8.83209059e-03, -5.78847875e-03, -1.46380996e-02,  4.35373450e-02,
        1.62704254e-02, -8.16206669e-02, -3.48791813e-02, -6.43317436e-02,
       -6.82033